In [ ]:
%%capture
!pip install antropy -q

### import  basic libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import statistics as st
from antropy import *

### read file

In [ ]:
df=pd.read_csv('data.csv')
df.head()

### seprate data and labels. the first column correponds to label

In [ ]:
data=df.iloc[:,2::].T
labels=df.iloc[:,1].T

#### print first five and last 5 rows of data and labels

In [ ]:
data.head()

In [ ]:
labels.head()

### Remove missing values columns
remove columns having missing/NaN values. Hint use pandas dropna. check dropna attributes:
how,inplace,axis

In [ ]:
data.dropna(how='any',inplace=True,axis=0)

In [ ]:
data.head()

In [ ]:
data.shape

### Plot the data 


In [ ]:
plt.plot(data.iloc[:,0:1].values,label='Baseline'); #PLOT 
plt.plot(data.iloc[:,27:28].values,label='Stressed');
plt.xlabel('time')
plt.ylabel('R-R interval')
plt.title('Raw data')
plt.legend()


### Remove Outliers
remove outliers, such as value greater than 1000 and lower than 600. you can either do it using 
pandas or numpy . you can use a better strategy such as inter quartile range

In [ ]:
data=np.where((data.values > 1000) | (data.values<600), np.median(data.values), data.values)


In [ ]:
plt.plot(data[:,0:1],label='Baseline'); #PLOT 
plt.plot(data[:,27:28],label='Stressed');
plt.xlabel('time')

plt.ylabel('R-R interval')
plt.title('Non outliered data')
plt.legend()

### Apply filter to the data
apply any smoothening filter such as Savitzky-Golay filter or moving average filter

In [ ]:
from scipy.signal import savgol_filter
data=savgol_filter(data,5,3)

In [ ]:
plt.plot(data[:,10:11],label='Non Filtered'); #PLOT 
plt.plot(data[:,10:11],label='Filtered');
plt.xlabel('time')
plt.ylabel('R-R interval')
plt.title('Savitzky-Golay filte data')
plt.legend()

## calculate features

calculate following features.
* mean
* median
* maximum
* variance
* standard deviation
* maximum
* minimum
* and the following

In [ ]:
def ranges(x):
    return x.max() - x.min()

def rmssd(x):
    return np.sqrt(np.mean(np.diff(x) ** 2))
def sdsd(x):
    return st.stdev(np.diff(x))
    
def nni_50(x):
    return  sum(np.abs(np.diff(x)) > 50)

def pnni_50(x):
    return 100 * nni_50(x) / len(x)

def nni_20(x):
    return sum(np.abs(np.diff(x)) > 20)

def pnni_20(x):
    return  100 * nni_20(x) / len(x)

def avg_hr(x):
    return  st.mean(60000/x)
def std_hr(x):
    return  st.stdev(60000/x)
def min_hr(x):
    return  min(60000/x)
def max_hr(x):
    return  max(60000/x)

def energy(x):
    return sum(np.square(x))

def abs_sum_diff(x):
#     sum of absolute differences (SAD) is a measure of the similarity between signal
    return sum(np.abs(np.diff(x)))




In [ ]:
data=pd.DataFrame(data)
time_features=data.agg([np.mean,np.var, np.median,np.max,np.min,
                   ranges,rmssd,sdsd,nni_50,pnni_50,nni_20,pnni_20,
                        avg_hr,std_hr,min_hr,max_hr,
                        energy,abs_sum_diff,

                       ],axis=0)

In [ ]:
time_features.head()

## Frequency features

In [ ]:
from scipy import signal
from scipy.ndimage import label
from scipy.stats import zscore
from scipy.interpolate import interp1d
from scipy.integrate import trapz

In [ ]:
data.shape,len(data)

In [ ]:
rr_interpolated=[]
for i in range(len(data)):
    rr_manual=data.T[i]
    x = np.cumsum(rr_manual) / 1000.0#cumulative sum of data
    f = interp1d(x, rr_manual, kind='cubic',fill_value="extrapolate")#extra polation 
    fs = 4.0#new sampling frequency
    steps = 1 / fs

    # now we can sample from interpolation function
    xx = np.arange(1, np.max(x), steps)
    rr_interpolated.append(f(xx))


In [ ]:
len(rr_interpolated),rr_interpolated[0].shape,rr_interpolated[27].shape,data.shape

In [ ]:
plt.plot(data.iloc[0],label='Non interploated Baseline'); #PLOT 
plt.plot(rr_interpolated[0],label='interploated Baseline');
plt.xlabel('time')
plt.ylabel('R-R interval')
plt.title('Interpolated data')
plt.legend()

In [ ]:
def frequency_domain(rri, fs=4):
    # Estimate the spectral density using Welch's method
    fxx, pxx = signal.welch(x=rri, fs=fs)
    
    '''
    Segement found frequencies in the bands 
     - Very Low Frequency (VLF): 0-0.04Hz 
     - Low Frequency (LF): 0.04-0.15Hz 
     - High Frequency (HF): 0.15-0.4Hz
    '''
    cond_vlf = (fxx >= 0) & (fxx < 0.04)
    cond_lf = (fxx >= 0.04) & (fxx < 0.15)
    cond_hf = (fxx >= 0.15) & (fxx < 0.4)
    
    # calculate power in each band by integrating the spectral density 
    vlf = trapz(pxx[cond_vlf], fxx[cond_vlf])
    lf = trapz(pxx[cond_lf], fxx[cond_lf])
    hf = trapz(pxx[cond_hf], fxx[cond_hf])
    
    # sum these up to get total power
    total_power = vlf + lf + hf

    # find which frequency has the most power in each band
    peak_vlf = fxx[cond_vlf][np.argmax(pxx[cond_vlf])]
    peak_lf = fxx[cond_lf][np.argmax(pxx[cond_lf])]
    peak_hf = fxx[cond_hf][np.argmax(pxx[cond_hf])]

    # fraction of lf and hf
    lf_nu = 100 * lf / (lf + hf)
    hf_nu = 100 * hf / (lf + hf)
    result=[vlf,lf,hf,total_power,lf/hf,peak_vlf,peak_lf,peak_hf,lf_nu,hf_nu]
    return np.array(result),fxx, pxx



In [ ]:
x=rr_interpolated[0]

In [ ]:
freq_feat=[]
for i in range(len(data.T)):
    results, fxx, pxx = frequency_domain(rr_interpolated[i])
    freq_feat.append(results)


In [ ]:
np.array(freq_feat).shape

In [ ]:
freq_col=['vlf','lf','hf','tot_pow','lf_hf_ratio','peak_vlf','peak_lf','peak_hf','lf_nu','hf_nu']
freq_features=pd.DataFrame(freq_feat,columns=freq_col)
freq_features.head()

In [ ]:
features=pd.concat([time_features.T,freq_features],axis=1)
features.head()

# Statistical tests

In [ ]:
from scipy import stats
t_test=stats.ttest_ind(features.iloc[0:len(features)//2],features.iloc[len(features)//2:len(features)])[1]

In [ ]:
t_test=pd.DataFrame(zip(features.columns.tolist(),t_test.tolist()),columns=['feature','p_value'])

In [ ]:
t_test[t_test['p_value']<0.05]
#t_test.round(3)

# Classification

### split features to train and test

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(features,labels,test_size=0.3,shuffle=True,stratify=labels)

### Scale the data

In [ ]:
from sklearn.preprocessing import StandardScaler,scale,MaxAbsScaler
scaling=StandardScaler()
X_train=scaling.fit_transform(X_train)
X_test=scaling.transform(X_test)

### Apply Classifier

In [ ]:
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression,LogisticRegressionCV
clf=SVC()
clf.fit(X_train,y_train)

In [ ]:
clf.score(X_test,y_test)

### Calculate predicted values

In [ ]:
y_pred=clf.predict(X_test)

### print classification report

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, y_pred))

In [ ]:







#feature=scale(features)
# clf=LogisticRegressionCV(max_iter=2000).fit(features,labels)
# clf.score(features,labels)

In [ ]:
feature=scale(features)
clf=LogisticRegressionCV(max_iter=2000).fit(feature,labels)
clf.score(feature,labels)

In [ ]:
from sklearn.model_selection import cross_val_score
cross_val_score(SVC(), feature,labels, cv = 10).mean()

# Classifiers

In [ ]:
from sklearn.metrics import accuracy_score, log_loss
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score,accuracy_score
from sklearn.pipeline import Pipeline
classifiers = [
    KNeighborsClassifier(),
    SVC(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    AdaBoostClassifier(),
    GradientBoostingClassifier(),
    GaussianNB(),
    LinearDiscriminantAnalysis(),
    QuadraticDiscriminantAnalysis(),
    LogisticRegression()]

In [ ]:
X,y=features,labels
accuracy=[]
accuracy_std=[]
pipeline = Pipeline([('transformer', StandardScaler()), ('estimator', clf)])

for clfs in classifiers:
    pipeline.set_params(estimator = clfs)
    name = clfs.__class__.__name__
    
    print("="*30)
    print(name)       
    print('****Results****')
    scores = cross_val_score(pipeline, X, y, cv=27)
    #f1_score = cross_val_score(clf, X_rfe, y, cv=5,scoring='f1')
    print("Accuracy: {:.4%}".format(np.array(scores).mean()))
    accuracy.append(np.array(scores).mean())
    accuracy_std.append(np.array(scores).std())


In [ ]:
import matplotlib.pyplot as plt
classifier=['KNN','SVC','DT','RF','Ada','GB','NB','LDA','QDA','LR']
y_pos = np.arange(len(classifier))
plt.bar(y_pos,np.array(accuracy))
ys=np.array(accuracy)
for index, value in enumerate(ys):
    plt.text(index-0.2,value-0.2, str(np.round(value,2)),rotation=90,color='white',fontsize=12)
plt.xticks(y_pos, classifier,fontsize=11)
plt.yticks(fontsize=11)
plt.ylabel('Accuracy',fontsize=12)
plt.xlabel('Classifiers',fontsize=12)
plt.title('Cross Validation Accuracy',fontsize=12)
plt.savefig('accuracy.svg',dip=300)